In [1]:
import os
import re
import json
from tqdm import tqdm
import pandas as pd
from bs4 import BeautifulSoup
from transformers import pipeline
from openai import AsyncOpenAI, OpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt

/Users/zgou/miniconda3/envs/SHIPLY_LLM/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
tqdm.pandas()

### Pure Embedding Approach for Topic Modeling

In [4]:
# 230 topics from BERTopic
selected_topics_df = pd.read_csv("./topics/embedding_similarity_label.csv")
selected_topics_df = selected_topics_df.drop(columns=['Unnamed: 0'])

In [5]:
selected_topics_df

,Topic,Name,Representation,closest_topic
0,-1,-1_schools_case_information_women,"['schools', 'case', 'information', 'women', 'p...",Personal Finance - Financial Assistance - Gove...
1,0,0_downtime_blah_salute_ok,"['downtime', 'blah', 'salute', 'ok', 'boy', 'n...",Communication
2,1,1_song_symphony_opera_blues,"['song', 'symphony', 'opera', 'blues', 'musici...",Fine Art - Opera
3,2,2_inning_yankees_red_francona,"['inning', 'yankees', 'red', 'francona', 'runs...",Sports - Baseball
4,3,3_candidates_primary_romney_gop,"['candidates', 'primary', 'romney', 'gop', 'we...",Politics - Elections
...,...,...,...,...
226,225,225_farms_csa_burpee_hirshberg,"['farms', 'csa', 'burpee', 'hirshberg', 'heron...",Business and Finance - Industries - Agriculture
227,226,226_crone_olympic_peterson_mantha,"['crone', 'olympic', 'peterson', 'mantha', 'li...",Sports - Olympic Sports - Winter Olympic Sports
228,227,227_saleh_yemeni_rabbo_taiz,"['saleh', 'yemeni', 'rabbo', 'taiz', 'marib', ...",Religion & Spirituality - Spirituality
229,228,228_detainees_waterboarding_conventions_geneva,"['detainees', 'waterboarding', 'conventions', ...",Science


In [7]:
# Load CURRENT GBH articles
#unseen_articles = pd.read_csv('./sample_data/se_naacp_db.articles_data.csv')
unseen_articles = pd.read_csv('./eval_dataset/Singer_Evaluation_Set.csv')
unseen_articles = unseen_articles.rename(columns={"body": "Body"})
unseen_articles = unseen_articles.dropna(subset=['Body'])
unseen_articles.reset_index(drop=True, inplace=True)

In [8]:
unseen_articles.columns

Index(['Singer notes ', '_id', 'neighborhoods[0]', 'neighborhoods[1]',
       'neighborhoods[2]', 'neighborhoods[3]', 'neighborhoods[4]',
       'neighborhoods[5]', 'position_section', 'tracts[0]', 'tracts[1]',
       'tracts[2]', 'tracts[3]', 'tracts[4]', 'tracts[5]', 'tracts[6]',
       'author', 'Body', 'content_id', 'hl1', 'hl2', 'pub_date', 'pub_name',
       'link', 'openai_labels[0]', 'Singer labels', 'userID', 'uploadID',
       'dateSum'],
      dtype='object')

In [9]:
# Load GBH articles
# unseen_articles = pd.read_csv('./sample_data/Articles Nov 2020 - March 2023.csv', usecols=range(12))
# unseen_articles = unseen_articles.dropna(subset=['Body'])
# unseen_articles = unseen_articles.sample(n=5000, random_state=1)
# unseen_articles.reset_index(drop=True, inplace=True)

### Calling of the OpenAI Embeddings

In [17]:
client = OpenAI(
    # This is the default and can be omitted
    api_key='',
)

In [18]:
# Retry up to 10 times with exponential backoff, starting at 1 second and maxing out at 20 seconds delay
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(10))
def get_embedding(text: str, model="text-embedding-ada-002"):
    #print(text)
    try:
        embedding = client.embeddings.create(input=text, model=model).data[0].embedding
        return embedding
    except Exception as e:
        print(f"Failed to retrieve ADA Embedding: {e}. Replacing with replacement value!")
        return [-1.0]
    return 

### Text Preprocessing

In [19]:
# News body preprocessing, gets rid of html tags
def truncate(tokens, length=500):
    """
    Function to get the first 500 elements from a list
    """
    return tokens[:length]

unseen_articles['Body'] = unseen_articles['Body'].apply(lambda x: re.sub(re.compile('<.*?>'), '', x))
unseen_articles['tokens'] = unseen_articles['Body'].apply(lambda x: x.split())
unseen_articles['tokens'] = unseen_articles['tokens'].apply(truncate)

print(max(len(row['tokens']) for _, row in unseen_articles.iterrows()))

unseen_articles['ada_embedding'] = unseen_articles.tokens.progress_apply(lambda x: get_embedding(','.join(map(str,x)), model='text-embedding-ada-002'))

500


 11%|███████████▉                                                                                               | 3/27 [00:00<00:01, 18.10it/s]

Failed to retrieve ADA Embedding: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-W4wZx***************************************otZu. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}. Replacing with replacement value!
Failed to retrieve ADA Embedding: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-W4wZx***************************************otZu. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}. Replacing with replacement value!
Failed to retrieve ADA Embedding: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-W4wZx***************************************otZu. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}. Re

 19%|███████████████████▊                                                                                       | 5/27 [00:00<00:01, 15.33it/s]

Failed to retrieve ADA Embedding: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-W4wZx***************************************otZu. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}. Replacing with replacement value!
Failed to retrieve ADA Embedding: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-W4wZx***************************************otZu. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}. Replacing with replacement value!


...

Failed to retrieve ADA Embedding: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-W4wZx***************************************otZu. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}. Replacing with replacement value!
Failed to retrieve ADA Embedding: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-W4wZx***************************************otZu. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}. Replacing with replacement value!
Failed to retrieve ADA Embedding: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-W4wZx***************************************otZu. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}. Re

 93%|██████████████████████████████████████████████████████████████████████████████████████████████████▏       | 25/27 [00:01<00:00, 13.84it/s]

Failed to retrieve ADA Embedding: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-W4wZx***************************************otZu. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}. Replacing with replacement value!
Failed to retrieve ADA Embedding: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-W4wZx***************************************otZu. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}. Replacing with replacement value!
Failed to retrieve ADA Embedding: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-W4wZx***************************************otZu. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}. Re

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:02<00:00, 13.30it/s]

Failed to retrieve ADA Embedding: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-W4wZx***************************************otZu. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}. Replacing with replacement value!
Failed to retrieve ADA Embedding: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-W4wZx***************************************otZu. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}. Replacing with replacement value!
Failed to retrieve ADA Embedding: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-W4wZx***************************************otZu. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}. Re

In [14]:
unseen_articles

,Singer notes,_id,neighborhoods[0],neighborhoods[1],neighborhoods[2],neighborhoods[3],neighborhoods[4],neighborhoods[5],position_section,tracts[0],...,pub_date,pub_name,link,openai_labels[0],Singer labels,userID,uploadID,dateSum,tokens,ada_embedding
0,it’s a GBH program,65aeb9b68d9d92f2327ea9d0,Allston,NaN,NaN,NaN,NaN,NaN,Local News,102,...,Mon Dec 18 10:45:00 EST 2023,GBH,https://www.wgbh.org/foundation/highlights/202...,Seafood Restaurants,NaN,org_2ZN4MA41LAA9l4j0rZBC5Olsr3Y,3f80ff3a-d5e6-4d09-bdbd-df10d3a87493,20231218,"[Tonight‚Äôs, celebrity, and, family-filled, p...",[-1.0]
1,Logan airport snow storm,65aeb9b68d9d92f2327ea9ce,East Boston,NaN,NaN,NaN,NaN,NaN,Local News,981300,...,Mon Dec 18 12:34:19 EST 2023,GBH,https://www.wgbh.org/news/local/2023-12-18/win...,Natural Disasters,NaN,org_2ZN4MA41LAA9l4j0rZBC5Olsr3Y,3f80ff3a-d5e6-4d09-bdbd-df10d3a87493,20231218,"[A, storm, making, its, way, up, the, East, Co...",[-1.0]
2,Nubian Sq block walk,65aeb9b68d9d92f2327eaa01,Roxbury,NaN,NaN,NaN,NaN,NaN,Local News,81800,...,Tue Nov 21 11:54:36 EST 2023,GBH,https://www.wgbh.org/news/local/2023-11-21/a-w...,Journalism and Media,Economic Development,org_2ZN4MA41LAA9l4j0rZBC5Olsr3Y,3f80ff3a-d5e6-4d09-bdbd-df10d3a87493,20231121,"[Paris, Alston:, This, is, GBH's, Morning, Edi...",[-1.0]
3,Maybe because United Way is in Fort Point?,65aeb9b68d9d92f2327eaa1f,South Boston Waterfront,NaN,NaN,NaN,NaN,NaN,Local News,61204,...,Tue Nov 07 16:40:47 EST 2023,GBH,https://www.wgbh.org/news/local/2023-11-07/mas...,Social Welfare,NaN,org_2ZN4MA41LAA9l4j0rZBC5Olsr3Y,3f80ff3a-d5e6-4d09-bdbd-df10d3a87493,20231107,"[As, the, state, nears, its, threshold, level,...",[-1.0]
4,Northeastern Univ,65adafef8d9d92f2327ea9a4,Fenway,Back Bay,NaN,NaN,NaN,NaN,Local News,10500,...,Thu Sep 21 14:13:53 EDT 2023,GBH,https://www.wgbh.org/news/health/2023-09-21/we...,Public Health,NaN,org_2bHDzl2Zax0nILIzDhui2DLWdH6,9394466f-107e-4a9e-85f5-c73a4f42b764,20230921,"[Northeastern, University, will, act, as, a, n...",[-1.0]
5,Boston Univ - Agganis Arena,65aeb9b68d9d92f2327ea9d1,Fenway,Allston,NaN,NaN,NaN,NaN,Local News,10103,...,Mon Dec 18 05:00:00 EST 2023,GBH,https://www.wgbh.org/news/local/2023-12-18/fac...,Charity Events,Sports,org_2ZN4MA41LAA9l4j0rZBC5Olsr3Y,3f80ff3a-d5e6-4d09-bdbd-df10d3a87493,20231218,"[For, 10, Sundays, each, year,, members, of, t...",[-1.0]
6,"No real clue why ""downtown""",65aeb9b68d9d92f2327eaa23,Downtown,South Boston,NaN,NaN,NaN,NaN,Local News,70102,...,Fri Nov 03 17:48:06 EDT 2023,GBH,https://www.wgbh.org/news/politics/2023-11-03/...,Political Issues & policy,NaN,org_2ZN4MA41LAA9l4j0rZBC5Olsr3Y,3f80ff3a-d5e6-4d09-bdbd-df10d3a87493,20231103,"[Deception, in, politics, is, a, tale, as, old...",[-1.0]
7,Mass & Cass,65aeb9b68d9d92f2327eaa13,Roxbury,NaN,NaN,NaN,NaN,NaN,Local News,80401,...,Mon Nov 13 12:06:44 EST 2023,GBH,https://www.wgbh.org/news/local/2023-11-13/wu-...,Law Enforcement,Housing/Homelessness,org_2ZN4MA41LAA9l4j0rZBC5Olsr3Y,3f80ff3a-d5e6-4d09-bdbd-df10d3a87493,20231113,"[Boston, Mayor, Michelle, Wu, said, late, last...",[-1.0]
8,Assume this is Gov office/Beacon Hill.,65aeb9b68d9d92f2327eaa27,Downtown,NaN,NaN,NaN,NaN,NaN,Local News,30302,...,Thu Nov 02 17:27:42 EDT 2023,GBH,https://www.wgbh.org/news/politics/2023-11-02/...,Criminal Justice,Housing/Homelessness,org_2ZN4MA41LAA9l4j0rZBC5Olsr3Y,3f80ff3a-d5e6-4d09-bdbd-df10d3a87493,20231102,"[Gov., Maura, Healey, says, she‚Äôs, bracing, ...",[-1.0]
9,East Boston mental health center,65aeb9b68d9d92f2327eaa18,East Boston,NaN,NaN,NaN,NaN,NaN,Local News,50901,...,Thu Nov 09 16:02:40 EST 2023,GBH,https://www.wgbh.org/news/health/2023-11-09/wi...,Health Insurance,Mental Health,org_2ZN4MA41LAA9l4j0rZBC5Olsr3Y,3f80ff3a-d5e6-4d09-bdbd-df10d3a87493,20231109,"[East, Boston, Neighborhood, Health, Center, h...",[-1.0]


### Ada Embedding -> Taxanomy List

In [ ]:
taxonomy_df = pd.read_csv('./taxonomy_list/Content_Taxonomy.csv', skiprows=5, usecols=range(8))
taxonomy_df.columns = taxonomy_df.iloc[0]
taxonomy_df = taxonomy_df.tail(-1)

In [ ]:
taxonomy_df

In [ ]:
tier_1_list = []
tier_2_list = []
tier_3_list = []
tier_4_list = []
for index, row in taxonomy_df.iterrows():
    if not pd.isnull(row['Tier 4']) and row['Tier 4'] != ' ':
        tier_1_label = row['Tier 1']
        tier_2_label = row['Tier 2']
        tier_3_label = row['Tier 3']
        tier_4_label = row['Tier 4']
        tier_4_list.append(f'{tier_1_label} - {tier_2_label} - {tier_3_label} - {tier_4_label}')
    elif not pd.isnull(row['Tier 3']) and row['Tier 3'] != ' ':
        tier_1_label = row['Tier 1']
        tier_2_label = row['Tier 2']
        tier_3_label = row['Tier 3']
        tier_3_list.append(f'{tier_1_label} - {tier_2_label} - {tier_3_label}')
    elif not pd.isnull(row['Tier 2']) and row['Tier 2'] != ' ':
        tier_1_label = row['Tier 1']
        tier_2_label = row['Tier 2']
        tier_2_list.append(f'{tier_1_label} - {tier_2_label}')
    else:
        tier_1_label = row['Tier 1']
        tier_1_list.append(f'{tier_1_label}')

tier_1_list = list(set(tier_1_list))
tier_2_list = list(set(tier_2_list))
tier_3_list = list(set(tier_3_list))
tier_4_list = list(set(tier_4_list))

tier_1_embedding = [get_embedding(topic) for topic in tier_1_list]
tier_2_embedding = [get_embedding(topic) for topic in tier_2_list]
tier_3_embedding = [get_embedding(topic) for topic in tier_3_list]
tier_4_embedding = [get_embedding(topic) for topic in tier_4_list]

all_topics_list = []
[all_topics_list.append(topic) for topic in tier_1_list]
[all_topics_list.append(topic) for topic in tier_2_list]
[all_topics_list.append(topic) for topic in tier_3_list]
[all_topics_list.append(topic) for topic in tier_4_list]

all_topics_embedding = []
[all_topics_embedding.append(embedding) for embedding in tier_1_embedding]
[all_topics_embedding.append(embedding) for embedding in tier_2_embedding]
[all_topics_embedding.append(embedding) for embedding in tier_3_embedding]
[all_topics_embedding.append(embedding) for embedding in tier_4_embedding]
print(len(all_topics_embedding))

### Similarity Metric Matching

In [ ]:
# Get embedding for the 230 topics selected by BERTopic 
selected_taxonomy_df = pd.read_csv('./topics/embedding_similarity_label.csv')
selected_taxonomy_df = selected_taxonomy_df.dropna(subset=['closest_topic'])
selected_topics_list = selected_taxonomy_df['closest_topic'].values.tolist()
# print(all_topics_list)

selected_topics_embedding = [get_embedding(topic) for topic in selected_topics_list]

In [ ]:
# Find most similar taxonomy to the BOW representations
closest_topic_list_all = []
for index, row in unseen_articles.iterrows():
    target_embedding = row['ada_embedding']
    similarities = [cosine_similarity(np.array(target_embedding).reshape(1, -1), np.array(topic).reshape(1, -1))[0][0] for topic in all_topics_embedding]

    # Find the index of the topic with the highest similarity
    closest_topic_index = np.argmax(similarities)

    # Retrieve the closest topic embedding
    closest_topic = all_topics_list[closest_topic_index]
    closest_topic_list_all.append(closest_topic)

unseen_articles['closest_topic_all'] = closest_topic_list_all
print(unseen_articles.head(10))

In [ ]:
# Find most similar taxonomy to the BOW representations
closest_topic_list_selected = []
for index, row in unseen_articles.iterrows():
    target_embedding = row['ada_embedding']
    similarities = [cosine_similarity(np.array(target_embedding).reshape(1, -1), np.array(topic).reshape(1, -1))[0][0] for topic in selected_topics_embedding]

    # Find the index of the topic with the highest similarity
    closest_topic_index = np.argmax(similarities)

    # Retrieve the closest topic embedding
    closest_topic = selected_topics_list[closest_topic_index]
    closest_topic_list_selected.append(closest_topic)

unseen_articles['closest_topic_selected'] = closest_topic_list_selected
print(unseen_articles.head(10))

### Export to CSV

In [ ]:
unseen_articles.to_csv('./outputs/pure_embedding_current_GBH_Data.csv')

### Quick Analysis

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Analysis on labels from two lists
selected_series = unseen_articles['closest_topic_selected']

# Get the value counts
selected_counts = selected_series.value_counts()
print(selected_counts)
# Identify categories with count 1
categories_to_combine = selected_counts[selected_counts <= 10].index

# Create a new category name for combined values
new_category_name = 'Other'

# Replace categories with count 1 with the new category
selected_series = selected_series.apply(lambda x: new_category_name if x in categories_to_combine else x)

# Create a pie chart
plt.figure(figsize=(22, 22))
plt.pie(selected_series.value_counts(), labels=selected_series.value_counts().index, autopct='%1.1f%%', startangle=140)
plt.title('Pie Chart of Selected Topics')
plt.show()


all_series = unseen_articles['closest_topic_all']

# Get the value counts
all_counts = all_series.value_counts()
print(all_counts)
# Identify categories with count 1
categories_to_combine = all_counts[all_counts <= 10].index

# Create a new category name for combined values
new_category_name = 'Other'

# Replace categories with count 1 with the new category
all_series = all_series.apply(lambda x: new_category_name if x in categories_to_combine else x)

# Create a pie chart
plt.figure(figsize=(22, 22))
plt.pie(all_series.value_counts(), labels=all_series.value_counts().index, autopct='%1.1f%%', startangle=140)
plt.title('Pie Chart of All Topics')
plt.show()

count_matching_rows = len(unseen_articles[unseen_articles['closest_topic_all'] == unseen_articles['closest_topic_selected']])
print(f'There are {count_matching_rows} matching labels in {len(unseen_articles)} articles. The ratio is {count_matching_rows/len(unseen_articles)*100}%.')